<a href="https://colab.research.google.com/github/mpondkule/datasets/blob/master/font_rec_deepfont_ergo_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from matplotlib.pyplot import imshow
import matplotlib.cm as cm
import matplotlib.pylab as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import PIL
from PIL import ImageFilter
import cv2
import itertools
import random
import keras
import imutils
from imutils import paths
import os
from keras import optimizers
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import callbacks
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D , UpSampling2D ,Conv2DTranspose
from keras import backend as K

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [ ]:
def pil_image(img_path):
    pil_im =PIL.Image.open(img_path).convert('L')
    pil_im=pil_im.resize((105,105))
    #imshow(np.asarray(pil_im))
    return pil_im

# Augumentation Steps
1) Noise
2) Blur
3) Perpective Rotation
4) Shading
5) Variable Character Spacing
6) Variable Aspect Ratio

In [ ]:
def noise_image(pil_im):
    # Adding Noise to image
    img_array = np.asarray(pil_im)
    mean = 0.0   # some constant
    std = 12  # some constant (standard deviation)
    noisy_img = img_array + np.random.normal(mean, std, img_array.shape)
    noisy_img_clipped = np.clip(noisy_img, 0, 255)
    noise_img = PIL.Image.fromarray(np.uint8(noisy_img_clipped)) # output
    #imshow((noisy_img_clipped ).astype(np.uint8))
    noise_img=noise_img.resize((105,105))
    return noise_img

In [ ]:
def blur_image(pil_im):
    #Adding Blur to image
    blur_img = pil_im.filter(ImageFilter.GaussianBlur(radius=3)) # ouput
    #imshow(blur_img)
    blur_img=blur_img.resize((105,105))
    return blur_img

In [ ]:
def affine_rotation(img):

    #img=cv2.imread(img_path,0)
    rows, columns = img.shape

    point1 = np.float32([[10, 10], [30, 10], [10, 30]])
    point2 = np.float32([[20, 15], [40, 10], [20, 40]])

    A = cv2.getAffineTransform(point1, point2)

    output = cv2.warpAffine(img, A, (columns, rows))
    affine_img = PIL.Image.fromarray(np.uint8(output)) # affine rotated output
    #imshow(output)
    affine_img=affine_img.resize((105,105))
    return affine_img


In [ ]:
def gradient_fill(image):
    #image=cv2.imread(img_path,0)
    laplacian = cv2.Laplacian(image,cv2.CV_64F)
    laplacian = cv2.resize(laplacian, (105, 105))
    return laplacian

## Preparing Dataset

In [ ]:
data_path = "/content/drive/MyDrive/font_patch_new_01/"
data=[]
labels=[]
imagePaths = sorted(list(paths.list_images(data_path)))
random.seed(42)
random.shuffle(imagePaths)

In [ ]:
def conv_label(label):
    if label == 'FSerProA-Bold':
        return 0
    elif label == 'FSerProA-Book':
        return 1
    elif label == 'FSerProA-Demi':
        return 2
    elif label == 'FSerProA-Medium':
        return 3
    elif label == 'FSMe-Bold':
        return 4
    elif label == 'FSMe-Light':
        return 5
    elif label == 'FSMe-Regular':
        return 6
    elif label == 'helvetica':
        return 7
    elif label == 'learning_curve_alt_G_bold_ot_tt':
        return 8
    elif label == 'learning_curve_altQ_altG_bold_ot_tt':
        return 9
    elif label == 'learning_curve_bold_ot_tt':
        return 10
    elif label == 'learning_curve_regular_ot_tt':
        return 11

In [ ]:
augument=["blur","noise","affine","gradient"]
a=itertools.combinations(augument, 4)

for i in list(a):
    print(list(i))

['blur', 'noise', 'affine', 'gradient']


In [ ]:
counter=0
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    label = conv_label(label)
    pil_img = pil_image(imagePath)
    #imshow(pil_img)

    # Adding original image
    org_img = img_to_array(pil_img)
    #print(org_img.shape)
    data.append(org_img)
    labels.append(label)

    augument=["noise","blur","affine","gradient"]
    for l in range(0,len(augument)):

        a=itertools.combinations(augument, l+1)

        for i in list(a):
            combinations=list(i)
            print(len(combinations))
            temp_img = pil_img
            for j in combinations:

                if j == 'noise':
                    # Adding Noise image
                    temp_img = noise_image(temp_img)

                elif j == 'blur':
                    # Adding Blur image
                    temp_img = blur_image(temp_img)
                    #imshow(blur_img)


                elif j == 'affine':
                    open_cv_affine = np.array(pil_img)
                    # Adding affine rotation image
                    temp_img = affine_rotation(open_cv_affine)

                elif j == 'gradient':
                    open_cv_gradient = np.array(pil_img)
                    # Adding gradient image
                    temp_img = gradient_fill(open_cv_gradient)

            temp_img = img_to_array(temp_img)
            data.append(temp_img)
            labels.append(label)

1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2
2
2
2
2
2
3
3
3
3
4
1
1
1
1
2


In [ ]:
data = np.asarray(data, dtype="float") / 255.0
labels = np.array(labels)
print("Success")
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

Success


In [ ]:
def replace_none_with_value(array, replacement_value):
    return [replacement_value if x is None else x for x in array]
replacement_value = 0
trainY = replace_none_with_value(trainY, replacement_value)
testY = replace_none_with_value(testY, replacement_value)
print(trainY)

[0, 3, 3, 7, 3, 6, 0, 0, 6, 9, 6, 2, 9, 1, 3, 10, 2, 10, 3, 10, 1, 4, 9, 0, 1, 11, 9, 3, 8, 4, 10, 7, 11, 5, 11, 9, 9, 10, 3, 11, 6, 10, 10, 1, 11, 11, 4, 2, 8, 9, 9, 2, 7, 3, 5, 8, 7, 6, 9, 10, 7, 9, 10, 8, 4, 9, 11, 5, 9, 5, 5, 9, 6, 11, 3, 9, 2, 7, 7, 8, 8, 0, 7, 8, 0, 10, 3, 2, 8, 5, 1, 6, 8, 3, 11, 6, 3, 2, 8, 0, 6, 3, 1, 9, 8, 5, 5, 9, 6, 2, 10, 7, 11, 1, 1, 9, 0, 1, 9, 10, 0, 7, 10, 10, 9, 9, 11, 9, 6, 9, 8, 9, 3, 2, 9, 0, 9, 2, 2, 11, 3, 5, 3, 0, 7, 0, 6, 2, 7, 2, 4, 11, 2, 4, 8, 2, 7, 1, 5, 4, 0, 4, 11, 11, 7, 7, 0, 8, 2, 11, 1, 5, 10, 3, 11, 11, 1, 7, 0, 3, 2, 1, 2, 1, 11, 0, 3, 7, 2, 2, 1, 3, 4, 0, 1, 5, 8, 1, 4, 7, 11, 7, 9, 1, 0, 1, 5, 5, 3, 9, 6, 6, 7, 11, 8, 6, 3, 2, 9, 6, 4, 1, 6, 0, 4, 0, 5, 3, 9, 4, 1, 6, 1, 5, 11, 5, 9, 6, 5, 3, 9, 5, 6, 5, 0, 11, 1, 7, 4, 2, 0, 1, 3, 7, 2, 8, 8, 10, 1, 7, 7, 2, 8, 6, 9, 4, 6, 1, 5, 7, 5, 5, 9, 7, 3, 4, 7, 10, 0, 8, 8, 6, 5, 5, 11, 10, 3, 11, 6, 11, 5, 8, 11, 7, 11, 4, 7, 9, 4, 11, 2, 7, 3, 10, 3, 9, 5, 2, 7, 10, 2, 3, 4, 2, 5, 6, 5,

In [ ]:
print(trainY, testY)

[0, 3, 3, 7, 3, 6, 0, 0, 6, 9, 6, 2, 9, 1, 3, 10, 2, 10, 3, 10, 1, 4, 9, 0, 1, 11, 9, 3, 8, 4, 10, 7, 11, 5, 11, 9, 9, 10, 3, 11, 6, 10, 10, 1, 11, 11, 4, 2, 8, 9, 9, 2, 7, 3, 5, 8, 7, 6, 9, 10, 7, 9, 10, 8, 4, 9, 11, 5, 9, 5, 5, 9, 6, 11, 3, 9, 2, 7, 7, 8, 8, 0, 7, 8, 0, 10, 3, 2, 8, 5, 1, 6, 8, 3, 11, 6, 3, 2, 8, 0, 6, 3, 1, 9, 8, 5, 5, 9, 6, 2, 10, 7, 11, 1, 1, 9, 0, 1, 9, 10, 0, 7, 10, 10, 9, 9, 11, 9, 6, 9, 8, 9, 3, 2, 9, 0, 9, 2, 2, 11, 3, 5, 3, 0, 7, 0, 6, 2, 7, 2, 4, 11, 2, 4, 8, 2, 7, 1, 5, 4, 0, 4, 11, 11, 7, 7, 0, 8, 2, 11, 1, 5, 10, 3, 11, 11, 1, 7, 0, 3, 2, 1, 2, 1, 11, 0, 3, 7, 2, 2, 1, 3, 4, 0, 1, 5, 8, 1, 4, 7, 11, 7, 9, 1, 0, 1, 5, 5, 3, 9, 6, 6, 7, 11, 8, 6, 3, 2, 9, 6, 4, 1, 6, 0, 4, 0, 5, 3, 9, 4, 1, 6, 1, 5, 11, 5, 9, 6, 5, 3, 9, 5, 6, 5, 0, 11, 1, 7, 4, 2, 0, 1, 3, 7, 2, 8, 8, 10, 1, 7, 7, 2, 8, 6, 9, 4, 6, 1, 5, 7, 5, 5, 9, 7, 3, 4, 7, 10, 0, 8, 8, 6, 5, 5, 11, 10, 3, 11, 6, 11, 5, 8, 11, 7, 11, 4, 7, 9, 4, 11, 2, 7, 3, 10, 3, 9, 5, 2, 7, 10, 2, 3, 4, 2, 5, 6, 5,

In [ ]:
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=12)
testY = to_categorical(testY, num_classes=12)

In [ ]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True)

In [ ]:
#K.set_image_dim_ordering('tf')
#K.image_data_format()
K.set_image_data_format('channels_last')

In [ ]:
def create_model():
  model=Sequential()

  # Cu Layers
  model.add(Conv2D(64, kernel_size=(48, 48), activation='relu', input_shape=(105,105,1)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(128, kernel_size=(24, 24), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2DTranspose(128, (24,24), strides = (2,2), activation = 'relu', padding='same', kernel_initializer='uniform'))
  model.add(UpSampling2D(size=(2, 2)))

  model.add(Conv2DTranspose(64, (12,12), strides = (2,2), activation = 'relu', padding='same', kernel_initializer='uniform'))
  model.add(UpSampling2D(size=(2, 2)))

  #Cs Layers
  model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))

  model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))

  model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))

  model.add(Flatten())

  model.add(Dense(4096, activation='relu'))

  model.add(Dropout(0.5))

  model.add(Dense(4096,activation='relu'))

  model.add(Dropout(0.5))

  model.add(Dense(2383,activation='relu'))

  model.add(Dense(12, activation='softmax'))

  return model

In [ ]:
batch_size = 512
epochs = 20
model= create_model()
#sgd = keras.optimizers.SGD(learning_rate=0.01,
 #   momentum=0.9,
#)
sgd = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

In [ ]:
early_stopping=callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')

filepath="top_model.keras"

checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [early_stopping,checkpoint]

In [ ]:
model.fit(trainX, trainY,shuffle=True,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(testX, testY),callbacks=callbacks_list)

Epoch 1/20
1/8 [==>...........................] - ETA: 1:59:11 - loss: 0.0781 - accuracy: 0.0918

In [ ]:
score = model.evaluate(testX, testY, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.07137015461921692
Test accuracy: 0.4621913433074951


In [ ]:
from keras.models import load_model
model = load_model('top_model.keras')

In [ ]:
score = model.evaluate(testX, testY, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.041123345494270325
Test accuracy: 0.5378086566925049


In [ ]:
def rev_conv_label(label):
    if label == 0 :
        return 'FSerProA-Bold'
    elif label == 1 :
        return 'FSerProA-Book'
    elif label == 2 :
        return 'FSerProA-Demi'
    elif label == 3 :
        return 'FSerProA-Medium'
    elif label == 4 :
        return 'FSMe-Bold'
    elif label == 5 :
        return 'FSMe-Light'
    elif label == 6 :
        return 'FSMe-Regular'
    elif label == 7 :
        return 'helvetica'
    elif label == 8 :
        return 'learning_curve_alt_G_bold_ot_tt'
    elif label == 9 :
        return 'learning_curve_altQ_altG_bold_ot_tt'
    elif label == 10 :
        return 'learning_curve_bold_ot_tt'
    elif label == 11 :
        return 'learning_curve_regular_ot_tt'

## get text layer on pdf

In [ ]:
pip install poppler-utils

In [ ]:
ls /content/drive/MyDrive/

'07 - Six'@                     form_1005.pdf
'07 - Six - Team only'@         notebooks/
 amzn_q2_output.pdf             Presentation.zip
 amzn_q2.pdf                   'python scripts'/
'Colab Notebooks'/             'Rohan_Sane_Resume 22nd  july 2016.doc'
'DL day 2 - Training NNs.mp4'   SampleInput.pdf
 documents/                     SampleOutput_02.pdf
'documents (1)'@                signature_output.pdf
'FB Teilnehmerbogen.docx'       signature.pdf
'Final projects workflow.mp4'   six_time_series_analysis.ipynb
 font_patch_new_01/            'Startups in Germany.gdoc'
 fonts/                        'Unbenanntes Jam.gjam'
 form_1005_output.pdf


In [ ]:
pip install pdf2image reportlab pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 40.2 MB/s eta 0:00:00


In [ ]:
import json
import os
import math
from pdf2image import convert_from_path
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, A4
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from pypdf import PdfReader, PdfWriter
from PIL import Image
from typing import Dict, List, Any, Optional

class BoundingBox():
    def __init__(self, left: float, bottom: float, right: float, top: float) -> None:
        self.bounds = [left, bottom, right, top]

    @classmethod
    def from_textract_bbox(cls, textract_bbox: Dict[str, float]) -> None:
        return cls(
            left=textract_bbox['Left'],
            bottom=textract_bbox['Top'] + textract_bbox['Height'],
            right=textract_bbox['Left'] + textract_bbox['Width'],
            top=textract_bbox['Top'],
        )

    def scale(self, x_scale: float, y_scale: Optional[float] = None) -> None:
        if not y_scale:
            y_scale = x_scale
        self.bounds[0] *= x_scale
        self.bounds[1] *= y_scale
        self.bounds[2] *= x_scale
        self.bounds[3] *= y_scale

    def __getitem__(self, key):
        return self.bounds[key]

    def __setitem__(self, key, value):
        self.bounds[key] = value

    @property
    def left(self) -> float:
        return self.bounds[0]

    @property
    def bottom(self) -> float:
        return self.bounds[1]

    @property
    def right(self) -> float:
        return self.bounds[2]

    @property
    def top(self) -> float:
        return self.bounds[3]

    @property
    def width(self) -> float:
        return abs(self.bounds[0] - self.bounds[2])

    @property
    def height(self) -> float:
        return abs(self.bounds[3] - self.bounds[1])

def convert_pdf_to_images(pdf_path: str, output_folder: str, dpi: int = 300, fmt: str = 'jpeg') -> List[str]:
    os.makedirs(output_folder, exist_ok=True)
    images = convert_from_path(pdf_path, dpi=dpi)
    image_paths = []
    for i, image in enumerate(images):
        image_path = os.path.join(output_folder, f"page_{i + 1}.{fmt}")
        image.save(image_path, fmt.upper())
        image_paths.append(image_path)
    return image_paths

def calculate_optimal_fontsize(bbox: BoundingBox, text: str, initial_fontsize: int, canvas_obj: canvas.Canvas, font: str) -> int:
    # Calculate based on width
    #text_width = canvas_obj.stringWidth(text, "Helvetica", initial_fontsize)
    text_width = canvas_obj.stringWidth(text, font, initial_fontsize)
    width_based_size = initial_fontsize * (bbox.width / text_width)

    # Calculate based on height (approximating that text height is 1.2 times the fontsize)
    height_based_size = bbox.height / 1.2

    # Use the smaller size to ensure text fits within both dimensions
    optimal_fontsize = max(width_based_size, height_based_size)
    return max(1, optimal_fontsize)
    #return max(1, int(round(width_based_size)))

def make_pdf_doc_searchable(
    pdf_path: str,
    textract_blocks: List[Dict[str, Any]],
    show_selectable_char: bool = False,
    verbose: bool = False,
) -> str:
    pdf_reader = PdfReader(pdf_path)
    images = Image.open('/content/drive/MyDrive/documents/form_1005.png')
    pdf_writer = PdfWriter()
    pdfmetrics.registerFont(TTFont('FSerProA-Bold', '/content/drive/MyDrive/fonts/FSerProA-Bold.ttf'))
    pdfmetrics.registerFont(TTFont('FSerProA-Book', '/content/drive/MyDrive/fonts/FSerProA-Book.ttf'))
    pdfmetrics.registerFont(TTFont('FSerProA-Demi', '/content/drive/MyDrive/fonts/FSerProA-Demi.ttf'))
    pdfmetrics.registerFont(TTFont('FSerProA-Medium', '/content/drive/MyDrive/fonts/FSerProA-Medium.ttf'))
    pdfmetrics.registerFont(TTFont('FSMe-Bold', '/content/drive/MyDrive/fonts/FSMe-Bold.ttf'))
    pdfmetrics.registerFont(TTFont('FSMe-Light', '/content/drive/MyDrive/fonts/FSMe-Light.ttf'))
    pdfmetrics.registerFont(TTFont('FSMe-Regular', '/content/drive/MyDrive/fonts/FSMe-Regular.ttf'))
    pdfmetrics.registerFont(TTFont('learning_curve_altQ_altG_bold_ot_tt', '/content/drive/MyDrive/fonts/learning_curve_altQ_altG_bold_ot_tt.ttf'))
    pdfmetrics.registerFont(TTFont('learning_curve_bold_ot_tt', '/content/drive/MyDrive/fonts/learning_curve_bold_ot_tt.ttf'))
    pdfmetrics.registerFont(TTFont('learning_curve_regular_ot_tt', '/content/drive/MyDrive/fonts/learning_curve_regular_ot_tt.ttf'))
    pdfmetrics.registerFont(TTFont('learning_curve_alt_G_bold_ot_tt', '/content/drive/MyDrive/fonts/learning_curve_alt_G_bold_ot_tt.ttf'))
    pdfmetrics.registerFont(TTFont('helvetica', '/content/drive/MyDrive/fonts/helvetica.ttf'))
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        #image = images[page_num]
        image_w, image_h = images.size
        print(image_w, image_h)
        media_box = page.mediabox
        x1, y1, x2, y2 = media_box
        width = float(x2 - x1)
        height = float(y2 - y1)

        img_temp = f"temp_page_{page_num + 1}.pdf"
        c = canvas.Canvas(img_temp, pagesize=(width, height))

        bbox_color = (220 / 255, 20 / 255, 60 / 255)
        fontsize_initial = 20
        print_step = 1000

        for blocki, block in enumerate(textract_blocks):
            if verbose and blocki % print_step == 0:
                print(f"Processing blocks {blocki} to {blocki + print_step} out of {len(textract_blocks)} blocks")
            if block["BlockType"] == "WORD":
                bbox = BoundingBox.from_textract_bbox(block["Geometry"]["BoundingBox"])
                bbox.scale(width, height)
                (x, y, w, h) = (block["Geometry"]["BoundingBox"]["Left"], block["Geometry"]["BoundingBox"]["Top"], block["Geometry"]["BoundingBox"]["Width"], block["Geometry"]["BoundingBox"]["Height"])
                #print(block["Geometry"]["BoundingBox"]["Left"])
                #print(block["Geometry"]["BoundingBox"]["Top"])
                #print(block["Geometry"]["BoundingBox"]["Width"])
                #print(block["Geometry"]["BoundingBox"]["Height"])

                text = block["Text"]
                # Crop the region from the image
                region = images.crop((x * image_w, y * image_h, (x + w) * image_w, (y + h) * image_h))
                #image_path = os.path.join(".", f"page_1.jpeg")
                #region.convert('RGB').save(image_path)
                #print(type(region))
                #pil_im =PIL.Image.open(region).convert('L')
                pil_im =region.convert('L')
                pil_im = blur_image(pil_im)
                org_img = img_to_array(pil_im)
                data=[]
                data.append(org_img)
                data = np.asarray(data, dtype="float") / 255.0
                y4 = model.predict(data)
                max_index = np.argmax(y4, axis=1)
                font_label = rev_conv_label(max_index)
                #y4 = np.round(y4).astype(int)
                #font_label = rev_conv_label(y4[0,0])
                #if text == "Salazar" or text == 'Carlos':
                #    font_label = 'learning_curve_alt_G_bold_ot_tt'
                optimal_fontsize = calculate_optimal_fontsize(bbox, text, fontsize_initial, c, font_label)

                c.setFillColorRGB(*bbox_color)
                #c.setFont("Helvetica", optimal_fontsize)
                c.setFont(font_label, optimal_fontsize)

                # Calculate text positioning to center within the bounding box
                text_x = bbox.left
                text_y = height - bbox.top - (bbox.height / 2) - (optimal_fontsize * 0.33)

                c.drawString(text_x, text_y, text)

        c.save()

        with open(img_temp, "rb") as f:
            temp_pdf_reader = PdfReader(f)
            temp_page = temp_pdf_reader.pages[0]
            page.merge_page(temp_page)
            pdf_writer.add_page(page)
        os.remove(img_temp)

    output_path = "/content/drive/MyDrive/documents/form_1005_output_ad_new_ML_01.pdf"
    with open(output_path, "wb") as output_pdf:
        pdf_writer.write(output_pdf)

    return output_path

# Main script
pdf_path = "/content/drive/MyDrive/documents/form_1005.pdf"
output_folder = "output_images"
#image_paths = convert_pdf_to_images(pdf_path, output_folder, dpi=300, fmt='jpeg')
data = json.load(open("/content/drive/MyDrive/documents/form_1005_ad.json"))
textract_blocks = data["Blocks"]
print(f"Number of blocks: {len(textract_blocks)}")
num_word_blocks = sum(1 for blk in textract_blocks if blk["BlockType"] == "WORD")
print(f"Number of WORD blocks: {num_word_blocks}")

searchable_pdf_path = make_pdf_doc_searchable(
    pdf_path=pdf_path,
    textract_blocks=textract_blocks,
    show_selectable_char=False,
    verbose=True,
)

print(f"Searchable PDF saved to: {searchable_pdf_path}")

Number of blocks: 1004
Number of WORD blocks: 663
4250 5500
Processing blocks 0 to 1000 out of 1004 blocks
1/1 [==============================] - 0s 35ms/step
Processing blocks 1000 to 2000 out of 1004 blocks
Searchable PDF saved to: /content/drive/MyDrive/documents/form_1005_output_ad_new_ML_01.pdf
